In [1]:
import numpy as np
from numba import jit
import time
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import math
from scipy.stats import norm

## Part II: Estimation of Sensitivities in MC

In [2]:
def Delta_Analytical(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1)

In [196]:
@jit(nopython=True, parallel=True)
def European_Put(T, K, r, S, sigma, trials):
    '''
    This function calculates the value of an European Put option
    Arguments: maturity, strike price, interest rate, stock price, volaility, number of trials
    Returns: Array of size trial with values of european puts
    '''
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)
    
    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        
        if K-S_cur > 0:
            payoff_array[i] = (K-S_cur)*np.exp(-r*T)
        else:
            payoff_array[i] = 0

    return payoff_array


def get_delta(kwargs, S, Se, seed=None):
    if seed:
        np.random.seed(seed)
    kwargs['S'] = S
    V = European_Put(**kwargs)
    kwargs['S'] = Se
    if seed:
        np.random.seed(seed)
    Ve = European_Put(**kwargs)
    
    return (Ve - V)/ (S-Se)

In [273]:
kwargs = {}
kwargs['T'] = 1
kwargs['K'] = 99
kwargs['r'] = 0.06
kwargs['sigma'] = 0.2
kwargs['trials'] = 10**6
S = 100
Se = 100.5
np.mean(get_delta(kwargs, S, Se, 100))

0.31875710283836844

In [66]:
Delta_Analytical(S, K, r, sigma, 1)

0.6737355117348961